In [1]:
import pandas as pd 
import numpy as np 
import dash
from dash import dcc, html, Input, Output, State, dash_table
import folium
from folium.plugins import MarkerCluster

In [2]:
resort=pd.read_csv('ski_resorts_with_coordinates.csv')
resort.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
def gen_google_link(row):
    latitude = row['Latitude']
    longitude = row['Longitude']
    return f'https://www.google.com/maps?q={latitude},{longitude}'

resort['Google Maps Link'] = resort.apply(gen_google_link, axis=1)

In [4]:
resort.head(5)

,Resort,Country,HighestPoint,LowestPoint,DayPassPriceAdult,BeginnerSlope,IntermediateSlope,DifficultSlope,TotalSlope,Snowparks,NightSki,SurfaceLifts,ChairLifts,GondolaLifts,TotalLifts,LiftCapacity,SnowCannons,Latitude,Longitude,Google Maps Link
0,Alpendorf (Ski amedé),Austria,1980,740,52,30,81,4,115,Yes,No,22,16,11,49,75398,600,47.321370,13.194450,"https://www.google.com/maps?q=47.32137,13.19445"
1,Soldeu-Pas de la Casa/​Grau Roig/​El Tarter/​C...,Andorra,2640,1710,47,100,77,33,210,Yes,Yes,37,28,7,72,99017,1032,42.532416,1.700219,"https://www.google.com/maps?q=42.5324162,1.700..."
2,Oberau (Wildschönau),Austria,1130,900,30,1,0,1,2,No,No,2,0,0,2,1932,0,47.437978,12.040429,"https://www.google.com/maps?q=47.4379785,12.04..."
3,Dachstein West,Austria,1620,780,42,15,33,3,51,Yes,Yes,25,8,3,36,32938,163,47.588769,13.462400,"https://www.google.com/maps?q=47.5887695,13.46..."
4,Rosa Khutor,Southern Russia,2320,940,22,30,26,21,77,Yes,No,6,11,10,27,49228,450,43.672786,40.297502,"https://www.google.com/maps?q=43.672786,40.297502"


In [36]:
# Create a Europe map
m = folium.Map(location=[51.5074, 7.1278], zoom_start=4) # The map centre in Europe
countries=resort['Country'].unique()
# Create the dashboard layout
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        options = countries,
        value=None,
        multi=True,
        id='Countries'
    ),
    html.Div(children=[html.Iframe(id='map',srcDoc=m._repr_html_(), width='100%', height=500)])
])

@app.callback(
    Output('map', 'srcDoc'),
    Input('Countries', 'value')
)
def update_map(Countries):
    if Countries is None:
        return m.get_root().render()
    else:
        # Add markers and popups
        resort_select=resort[resort['Country'].isin(list(Countries))]
        marker_cluster = MarkerCluster().add_to(m)
        for index, row in resort_select.iterrows():
            popup = f"<b>{row['Resort']}</b><br><a href='{row['Google Maps Link']}' target='_blank'>Google Map Link</a>"
            folium.Marker([row['Latitude'], row['Longitude']], 
                          popup=folium.Popup(max_width=100,html=popup),
                          icon=folium.Icon(icon="info-sign")).add_to(marker_cluster)
        return m.get_root().render()


if __name__ == '__main__':
    app.run_server(debug=True)